<a href="https://colab.research.google.com/github/DAOTHITHUNGA/CS114.K21.KHTN/blob/master/lab7_phanloaivanban.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
from sklearn.model_selection import train_test_split 
from sklearn.naive_bayes import MultinomialNB as MNB
from sklearn.neighbors import KNeighborsClassifier as KNC
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import tarfile
#from sklearn.datasets import fetch_20newsgroups
import re
import pickle

#Load data

##From zip file

In [ ]:
data_path = '/content/drive/MyDrive/truyvan/20_newsgroups'
def load_data(data_path):
    x_data = []
    y_data = []
    file_name = []
    labels = os.listdir(data_path)
    for label in labels:
        print(f'Reading {label}...')
        folder = os.path.join(data_path,label)
        files = os.listdir(os.path.join(folder))
        for i,file in enumerate(files):
            with open(os.path.join(folder,file),errors ='ignore', encoding = 'utf-8') as data:
                #text là đọc ghi lại giá trị của x x_data.append
                text = data.read()
                x_data.append(text)
                #y chứa label 
                y_data.append(label)
                file_name.append(file)
                if (i+1)%100 ==0:
                    print(f'\tRead {i+1} files.')
    return x_data, y_data, labels,file_name
x_data,y_data,labels,file_name = load_data(data_path) 
print(len(x_data))


Reading rec.sport.hockey...
	Read 100 files.
	Read 200 files.
	Read 300 files.
	Read 400 files.
	Read 500 files.
	Read 600 files.
	Read 700 files.
	Read 800 files.
	Read 900 files.
	Read 1000 files.
Reading sci.crypt...
	Read 100 files.
	Read 200 files.
	Read 300 files.
	Read 400 files.


##preprocessing

In [ ]:
def preprocessing(text_data):

    ps = PorterStemmer()
    res = []
    for i in range(len(text_data)):
        text = text_data[i]
        text = re.sub('[^a-z]', ' ', text_data[i].lower())
        #text = text.apply(lambda s : re.sub('[^a-zA-Z]', ' ', s))
        res.append( ' '.join([ps.stem(word) for word in text.split()]))
        #text = text.apply(lambda x: x.split())
        #text = text.apply(lambda x : " ".join([ps.stem(word) for word in x]))
    return res
def vectorize(train,test):
    vectorizer1 = CountVectorizer(analyzer='word',ngram_range=(1,1), max_features=10000) 
    features_train = vectorizer1.fit_transform(train)

    vectorizer2 = CountVectorizer(vocabulary=vectorizer1.vocabulary_,ngram_range=(1,1)) 
    features_test = vectorizer2.fit_transform(test)
    return features_train, features_test

#import TfidfVectorizer
#Một thuật toán rất phổ biến để chuyển đổi văn bản thành một đại diện có ý nghĩa của các con số được biểu diễn dưới dạng vector
#from sklearn.feature_extraction.text import TfidfVectorizer
#tv = TfidfVectorizer()
#tv.fit(headline_kaggle)
#bag_of_word  = tv.vocabulary_
#in ra so tu moi chua trong thu vien 
#print(bag_of_word)
#print(len(bag_of_word))
#def feature_extract(headlines):
#   tv = TfidfVectorizer(vocabulary=bag_of_word, max_features=5000)
#  headlines = list(headlines)
#    features = tv.fit_transform(headlines).toarray()
#    return features


SAVE FEATURES + LABELS

In [ ]:
# Đường dẫn chứa các file lưu
%cd '/content/drive/MyDrive/truyvan/'

#FEATURES
# Tên file lưu là 20NGs_Feature
featuresPickle = open('20NGs_Feature', 'wb') 
# Lưu x_data vào file vừa đặt tên
pickle.dump(x_data, featuresPickle)

In [ ]:
#LABELS
%cd '/content/drive/MyDrive/truyvan/'
labelsPickle = open('labels', 'wb')
pickle.dump(y_data, labelsPickle)

LOAD SAVED FEATURES + LABELS FILES

In [ ]:
x_data=pickle.load(open('/content/drive/MyDrive/truyvan/20NGs_Feature','rb'))
y_data=pickle.load(open('/content/drive/MyDrive/truyvan/labels','rb'))

In [ ]:
print(len(x_data),len(y_data))

SPLIT DATA

In [ ]:
x_data = preprocessing(x_data)
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.25,shuffle= True, random_state=56)

In [ ]:
def training(x_train,y_train,x_test,y_test, n_samples = 10000, max_features = 2000):
    samples = random.sample(range(len(x_train)),n_samples)
    X_train = [x_train[i] for i in samples]
    Y_train = [y_train[i] for i in samples]

    naiB = MNB() #Naive Bayes
    knn = KNC() #KNeighborsClassifier
#train model 
    vectorizer1 = CountVectorizer(analyzer='word',ngram_range=(1,1), max_features=max_features) 
    features_train = vectorizer1.fit_transform(X_train) 
#test model 
    vectorizer2 = CountVectorizer(vocabulary=vectorizer1.vocabulary_,ngram_range=(1,1)) 
    features_test = vectorizer2.fit_transform(x_test)

    naiB.fit(features_train,Y_train)
    naiB_train = naiB.score(features_train,Y_train)
    naiB_score = naiB.score(features_test,y_test)
    print(n_samples)
    print("MNB: ",naiB_score)

    knn.fit(features_train,Y_train)
    knn_train = knn.score(features_train,Y_train)
    knn_score = knn.score(features_test,y_test)
    print("KNN: ",knn_score)


    return mnb_train, naiB_score, knn_train, knn_score

#plot result

In [ ]:
list_n_task = [1000, 3000, 5000, 7000, 9000, 11000,13000, len(x_train)]
mnb_train = []
mnb_test  = []
knn_train = []
knn_test  = []

for n_task in list_n_task:
    res = training(x_train,y_train,x_test,y_test,n_task)


1000
MNB:  0.6758
KNN:  0.3222
3000
MNB:  0.7622
KNN:  0.4502
5000
MNB:  0.78
KNN:  0.4816
7000
MNB:  0.7962
KNN:  0.522
9000
MNB:  0.799
KNN:  0.5548
11000
MNB:  0.7958
KNN:  0.578
13000
MNB:  0.8012
KNN:  0.602
14997
MNB:  0.8056
KNN:  0.6094
